In [1]:
import pandas as pd
import keras
from keras.models import load_model
import numpy as np

/Users/alex/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model_path = './models/basic_regression_model_beta.h5'

df = pd.read_csv('./data/csv/test_acc10.csv')
model = load_model(model_path)

In [3]:
df.head()

,dest,lig_id,pro_id,score
0,./data/processed_data/test_acc10/2701_pro_2701...,2701,2701,1
1,./data/processed_data/test_acc10/2702_pro_2701...,2701,2702,0
2,./data/processed_data/test_acc10/2703_pro_2701...,2701,2703,0
3,./data/processed_data/test_acc10/2704_pro_2701...,2701,2704,0
4,./data/processed_data/test_acc10/2705_pro_2701...,2701,2705,0


In [4]:
# group by ligand
# for each group, compute top10 accuracy manually

matches = 0
total = float(len(df.lig_id.unique()))
debug = False

for lig_id, grp in df.groupby('lig_id'):
    grp.reset_index(inplace=True)
    
    match_id = grp.index[grp['pro_id']==lig_id]
    
    # load all features
    n_channels = 16
    dims = (24,24,24)
    X = np.empty((len(grp.index), *dims, n_channels))
    for row in grp.itertuples():
        X[row[0],] = np.load(row.dest)
        
    probabilities = model.predict_on_batch(X)
    probabilities = probabilities.flatten()
    
    largest_first = reversed(np.argsort(probabilities).tolist())
    top10 = list(largest_first)[:10]
    
    if debug:
        print('ligid: ', lig_id)
        print('match_id: ', match_id)
        print('top_10: ', top10)
        print()
    
    if match_id in top10:
        matches += 1
        
print(matches/total)    

1.0
